# Linear regression in PyTorch
Here the linear regression model is implemented using PyTorch

In [1]:
import numpy as np
import torch

In the example the objective is to predict the yields for oranges and apples given the values of temperature, rainfall and humidity. Therefore the (linear) models are going to be
$$yield_{apple}  = w_{11} * \text{temp} + w_{12} * \text{rainfall} + w_{13} * \text{humidity} + b_{1}$$
$$yield_{apple}  = w_{21} * \text{temp} + w_{22} * \text{rainfall} + w_{23} * \text{humidity} + b_{2}$$

## Input data
The input data are the following

In [3]:
# Input (temp, rainfall, humidity)
inputs = np.array(
    [[73, 67, 43],
    [91, 88, 64],
    [87, 134, 58],
    [102, 43, 37],
    [69, 96, 70]], 
    dtype='float32'
)

inputs

array([[ 73.,  67.,  43.],
       [ 91.,  88.,  64.],
       [ 87., 134.,  58.],
       [102.,  43.,  37.],
       [ 69.,  96.,  70.]], dtype=float32)

The targets (labels) are the following

In [4]:
# Targets (apples, oranges)
targets = np.array(
    [[56, 70],
    [81, 101],
    [119, 133],
    [22, 37],
    [103, 119]], 
    dtype='float32'
)

targets

array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)

Transform the input and targets in tensors

In [5]:
inputs = torch.from_numpy(inputs)
inputs

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])

In [6]:
targets = torch.from_numpy(targets)
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

## Weights and biases initialization
Weights and biases are initialized with random values following a normal distribution with mean 0 and standard deviation 1. 
- Weights are collected in a tensor having the where the first dimension (rows) corresponds to the number of models (2, i.e., the number of outputs) and the second dimension (columns) corresponds to the number of features (3, i.e., the number of inputs)
- Biases are collected in a tensor having the first dimension equal to the number of models (2)

In [13]:
w = torch.randn((2,3), requires_grad=True)
w

tensor([[-0.4362,  0.0947, -0.7485],
        [ 0.5128,  1.2666,  0.3148]], requires_grad=True)

In [14]:
b = torch.randn((2), requires_grad=True)
b

tensor([-1.4410,  0.7586], requires_grad=True)

## The model

The model is 
$$X \cdot W^{T} + B$$
This model can be written in python as a function of <code>x</code>

In [17]:
def regression_model(x):
    return x @ w.T + b

where 
- <code>@</code> is the matrix multiplication operator
- <code>.T</code> (or <code>.t()</code>) outputs the transpose tensor

Moreover, notice that <code>b</code> is a vector, while the result of <code>x @ w.T</code> is a matrix: the summation is computed only after <code>b</code> is broadcasted on the rows to match the matrix shape

## The predictions
The predictions of the model are computed calling the function

In [18]:
preds = regression_model(inputs)
preds

tensor([[-59.1214, 136.5940],
        [-80.7021, 179.0344],
        [-70.1084, 233.3587],
        [-69.5532, 119.1782],
        [-74.8394, 179.7741]], grad_fn=<AddBackward0>)

If the predictions are compared with the targets it can be seen that they are quite different. That's because the model has not been trained yet, and weights and biases still have random values

## Definition of a Loss function

Improving the model, it must be evaluated how well the model is performing. The model predictions are compared with the actual targets using the following method:

- Calculate the difference between the two matrices (`preds` and `targets`).
- Square all elements of the difference matrix to remove negative values.
- Calculate the average of the elements in the resulting matrix.

The result is a single number, known as the **mean squared error** (MSE).
$$\frac{\sum{(\text{pred}-\text{target})^{2}}}{N}$$

In [26]:
def mse(t1, t2):
    diff = t1 - t2
    quad_diff = diff * diff
    mean_quad_diff = torch.sum(quad_diff) / diff.numel()
    return mean_quad_diff

So the loss, in this case, is

In [27]:
mse_loss = mse(preds, targets)

## Gradient computation

Since <code>w</code> and <code>b</code> have been defined with <code>requires_grad</code> equal to True, the gradient of the loss function is going to be computed w.r.t <code>w</code> and <code>b</code>

In [28]:
mse_loss.backward()

The gradients are stored in the <code>.grad</code> attribute of <code>w</code> and <code>b</code>

In [30]:
w.grad, b.grad

(tensor([[-12236.1045, -13658.5625,  -8420.7324],
         [  6653.8569,   6828.9727,   4194.6650]]),
 tensor([-147.0649,   77.5879]))

### Adjust weights and biases to reduce the loss

The loss is a [quadratic function](https://en.wikipedia.org/wiki/Quadratic_function) of weights and biases, and the objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like the figure shown below. An important insight from calculus is that the gradient indicates the rate of change of the loss, i.e., the loss function's [slope](https://en.wikipedia.org/wiki/Slope) w.r.t. the weights and biases.

If a gradient element is **positive**:

- **increasing** the element weight value slightly will **increase** the loss
- **decreasing** the element weight value slightly will **decrease** the loss

![postive-gradient](https://i.imgur.com/WLzJ4xP.png)

If a gradient element is **negative**:

- **increasing** the element weight value slightly will **decrease** the loss
- **decreasing** the element weight value slightly will **increase** the loss

![negative=gradient](https://i.imgur.com/dvG2fxU.png)

The increase or decrease in the loss by changing a weight element is proportional to the gradient of the loss w.r.t. that element. This observation forms the basis of _the gradient descent_ optimization algorithm that we'll use to improve our model (by _descending_ along the _gradient_).

We can subtract from each weight element a small quantity proportional to the derivative of the loss w.r.t. that element to reduce the loss slightly.

Since PyTorch accumulates gradients, to procede the gradients of <code>w</code> and <code>b</code> have to be reset to 0

In [33]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


Then, we register the current loss to compare it later

In [35]:
old_mse_loss = mse(preds, targets)